In [ ]:
import re
import csv

t = open('/home/faculty/millbr02/pub/un-debates-augmented-1.csv','r',encoding='latin-1')
csv_iter = csv.reader(t)
stupid_list = []
db = []
for row in csv_iter:
        db.append(row)
        stupid_list.append(row[4])

In [ ]:
import psycopg2
conn = psycopg2.connect("postgresql://millbr02:@localhost/world")
curs = conn.cursor()
curs.execute("select * from country limit 400")
res = curs.fetchall()
stupid_dict1 = {}
for row in res:
    stupid_list.append(row[1])

In [ ]:
stupid_matrix = {x: {} for x in stupid_list}
for k in stupid_matrix:
    stupid_matrix[k] = {x:0 for x in stupid_list}

In [ ]:
for row in db[1:]:
    speech = row[3]
    countryspeaking = row[4]
    for word in stupid_list:
        count = speech.count(word)
        stupid_matrix[countryspeaking][word] += count

In [ ]:
mentions = {}
for key in stupid_matrix:
    for key2 in stupid_matrix[key]:
        if key not in mentions:
            mentions[key] = [int(stupid_matrix[key][key2])]
        else:
            mentions[key].append(int(stupid_matrix[key][key2]))

In [ ]:
totalcount = {}        
for key in mentions:
    count = sum(mentions[key])
    totalcount[key] = count

In [ ]:
import requests
res = requests.get("https://opendata.socrata.com/api/views/mnkm-8ram/rows.json?accessType=DOWNLOAD")
j = res.json()
lat_long={}
for item in j['data']:
    lat_long[item[10]] = {'Lat':item[12], 'Long':item[13]}

In [ ]:
for line in db:
    for key in lat_long:
        if line[2] == key:
            line.insert(3,('Latitude:', lat_long[key]['Lat']))
            line.insert(4, ('Longitude: ', lat_long[key]['Long']))
    for k in totalcount:
        if line[6] == k:
            line.insert(6, ('United Nations Mentions (1970 - 2008): ', str(totalcount[k])))

In [ ]:
final_list = []
for line in db:
    if len(line) < 9:
        del line[3]
    del line[0]
    del line[3]
    line[1] = ''.join(line[1])
    line[2] = ''.join(line[2])
    if len(line) > 3:
        line[3] = ''.join(line[3])
    final_list.append(line)

In [ ]:
no_doubles = []
actual_final_list = []
for line in final_list:
    if line[0] not in no_doubles:
        no_doubles.append(line[0])
        actual_final_list.append(line)
    else:
        pass

In [ ]:
actual_final_list[0] = ['Country code', 'Latitude', 'Longitude', 'Total mentions', 'Country name', 'Region', 'Continent']
actual_final_list